# Creating EVT and BPS Symbology

1. Load the CSV file

In [4]:
import xml.etree.ElementTree as ET
import csv
import os
bps_root = ET.parse('us_200bps.tif.aux.xml').getroot()
evt_root = ET.parse('us_200evt.tif.aux.xml').getroot()
os.makedirs('output', exist_ok=True)

In [5]:
def sort_and_write(filename, rows):
  """Here's a little helper function to write our .clr file once we have the format

  Args:
      filename ([type]): [description]
      rows ([type]): This is going to be a list of lists in the form: [[VAL(float), R(int),G(int),B(int), Label(str)], ...]
  """
  # Now sort by value column
  rows_sorted = sorted(rows, key=lambda k : k[0])

  # Open up a csvfile for writing as a TSV (Tab delimited)
  with open(os.path.join('output', filename), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    # Write the file
    for r in rows_sorted:
      writer.writerow(r)

SyntaxError: invalid syntax (2046266001.py, line 12)

In [ ]:
# Extract the field definitions and the values
bps_defns_arr = bps_root.findall('PAMRasterBand/GDALRasterAttributeTable/FieldDefn')
bps_defns = [{c.tag:c.text for c in r.findall('*') } for r in bps_defns_arr]

bps_rows_arr = bps_root.findall('PAMRasterBand/GDALRasterAttributeTable/Row')
bps_rows = [{bps_defns[i]['Name']:c.text for i,c in enumerate(r.findall('*')) } for r in bps_rows_arr]

evt_defns_arr = evt_root.findall('PAMRasterBand/GDALRasterAttributeTable/FieldDefn')
evt_defns = [{c.tag:c.text for c in r.findall('*') } for r in evt_defns_arr]

evt_rows_arr = evt_root.findall('PAMRasterBand/GDALRasterAttributeTable/Row')
evt_rows = [{evt_defns[i]['Name']:c.text for i,c in enumerate(r.findall('*')) } for r in evt_rows_arr]

In [ ]:
# Now make an array of rows for the symbologies we want
def write_clr(value_col, label_col, ds):
    rows = []
    for r in ds:
      r_val = r['r'] if 'r' in r else r['R']
      g_val = r['g'] if 'g' in r else r['G']
      b_val = r['b'] if 'b' in r else r['B']
      rows.append([r[value_col], r_val, g_val, b_val, '{}: {}'.format(r[value_col], r[label_col])])
    return rows

evt_names = write_clr('VALUE', 'EVT_Name', evt_rows)
bps_names = write_clr('VALUE', 'BPS_NAME', bps_rows)

sort_and_write('evt_names.clr', evt_names)
sort_and_write('bps_names.clr', bps_names)


## Existing_Veg_EVT_Class 

This one needs a little extra care because it needs a custom palett

In [ ]:
# Load the color lookup from the JSON file
import json

def veg_class():
  with open('evt_class_lookup.json', 'r') as f:
    evt_class_lookup = json.load(f)

  evt_classes = []
  rows = []
  for r in evt_rows:
    if r['EVT_CLASS'] not in evt_classes:
      evt_classes.append(r['EVT_CLASS'])
    if r['EVT_CLASS'] not in evt_class_lookup:
      raise Exception('Could not find class "{}" in lookup'.format(r['EVT_CLASS']))

    rows.append([r['VALUE'], *evt_class_lookup[r['EVT_CLASS']], r['EVT_CLASS']])

  # Here are the keys from the JSON
  print("Lookup Values", evt_class_lookup.keys(), len(evt_class_lookup.keys()))

  # Here are the unique values we found in the RAT
  print("RAT Values", evt_classes, len(evt_classes))

  return rows

sort_and_write('evt_classes.clr', veg_class())



Lookup Values dict_keys(['Sparse tree canopy', 'Closed tree canopy', 'Herbaceous - shrub-steppe', 'Open tree canopy', 'Herbaceous - grassland', 'Shrubland', 'Sparsely vegetated', 'Dwarf-shrubland', 'No Dominant Lifeform', 'Non-vegetated', 'Open Tree Canopy']) 11
RAT Values ['Sparse tree canopy', 'Closed tree canopy', 'Herbaceous - shrub-steppe', 'Open tree canopy', 'Herbaceous - grassland', 'Shrubland', 'Sparsely vegetated', 'Dwarf-shrubland', 'No Dominant Lifeform', 'Non-vegetated', 'Open Tree Canopy'] 11


## Existing_Veg_Riparian EVT

For this one we need to do a test to see if the column has the label 'Riparian' or not

In [ ]:
riparian_row = [0,255,0, 'Riparian']
non_rip_row = [255,255,255, 'Non-Riparian']

def riparian_test(ds, val_col, test_col, test_val):
  rows = []
  for r in ds:
    if r[test_col] == test_val:
      rows.append([r[val_col], *riparian_row])
    else:
      rows.append([r[val_col], *non_rip_row])
  return rows

evt_riparian = riparian_test(evt_rows, 'VALUE', 'EVT_PHYS', 'Riparian')
bps_riparian = riparian_test(bps_rows, 'VALUE', 'GROUPVEG', 'Riparian')

sort_and_write('EVT_Riparian.clr', evt_riparian)
sort_and_write('BPS_Riparian.clr', bps_riparian)
